## 일베 댓글+ 캐글 데이터로 fast-text 임베딩 하기

In [29]:
import pandas as pd
import numpy as np
import re

from gensim.models import FastText
from sklearn.model_selection import train_test_split

from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.python.keras.layers import Embedding, Dense, LSTM
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
df = pd.read_csv('./비속어댓글총합_특수문자,null값제거o.csv')
df

,댓글,비속어여부
0,좌배 까는건,1
1,집에 롱 패딩만 세 개다 10년 더 입어야지,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고,1
...,...,...
190576,드라마도 완전 재밌고 배우들도 멋있어서,0
190577,원작을 읽을 때 이런 건 절대 영상화하기 힘들다고 생각했는데 벤휘쇼의 연기와 더불어...,0
190578,케석대 어깨 올라간거봐라,1
190579,로버트다우니주니어를 좋아해서 봤는데너무재밌게 봤던영화생각없이 볼때 딱좋음,0


In [4]:
# # 문장 내 한글자만 이루어진 댓글은 삭제

# def onewordremove(sentence):
#     sentence_list = []
#     for index in range(len(sentence.split())):
#         if len(sentence.split()[index]) > 1: #문자 1글자 초과일 때
#             sentence_list.append(sentence.split()[index]) #문자 1글자로 이루어진 단어만 빼고 sentence_list에 추가
#     sentence = " ".join(sentence_list) #리스트의 내용을 하나의 문자열로 join
#     return sentence

# df['댓글'] = df['댓글'].apply(onewordremove)

In [5]:
tmp = df.copy()

In [59]:
df.head(10)

,댓글,비속어여부
0,좌배 까는건,1
1,집에 롱 패딩만 세 개다 10년 더 입어야지,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고,1
5,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,1
6,재앙이한건햇노,1
7,글쓴이 와꾸 승리에 비하면 방사능 피폭 원숭이 일듯,1
8,마 씨발련 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나,1
9,은행에 대출 상담 받으러 가보면 직업의 귀천 바로 알려줌,0


# 자모 분류, fast text 는 git 참고 
https://github.com/smothly/bad-word-detection/blob/master/FastTextVocab.ipynb

In [42]:

CHOSUNGS = [u'ㄱ',u'ㄲ',u'ㄴ',u'ㄷ',u'ㄸ',u'ㄹ',u'ㅁ',u'ㅂ',u'ㅃ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅉ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
JOONGSUNGS = [u'ㅏ',u'ㅐ',u'ㅑ',u'ㅒ',u'ㅓ',u'ㅔ',u'ㅕ',u'ㅖ',u'ㅗ',u'ㅘ',u'ㅙ',u'ㅚ',u'ㅛ',u'ㅜ',u'ㅝ',u'ㅞ',u'ㅟ',u'ㅠ',u'ㅡ',u'ㅢ',u'ㅣ']
JONGSUNGS = [u'_',u'ㄱ',u'ㄲ',u'ㄳ',u'ㄴ',u'ㄵ',u'ㄶ',u'ㄷ',u'ㄹ',u'ㄺ',u'ㄻ',u'ㄼ',u'ㄽ',u'ㄾ',u'ㄿ',u'ㅀ',u'ㅁ',u'ㅂ',u'ㅄ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
TOTAL = CHOSUNGS + JOONGSUNGS + JONGSUNGS

# 자모분리
def jamo_split(word, end_char="_"):
    
    result = []
    
    for char in word:
        
        character_code = ord(char)
        
        if 0xD7A3 < character_code or character_code < 0xAC00:
            result.append(char)
            continue

        chosung_index = int((((character_code - 0xAC00) / 28) / 21) % 19)
        joongsung_index = int(((character_code - 0xAC00) / 28) % 21)
        jongsung_index = int((character_code - 0xAC00) % 28)
        
        chosung = CHOSUNGS[chosung_index]
        joongsung = JOONGSUNGS[joongsung_index]
        jongsung = JONGSUNGS[jongsung_index]
        
        # 종성 범위 밖에 있는 것들은 end_char로 메꿔준다.
        if jongsung_index == 0:
            jongsung = end_char
        
        result.append(chosung)
        result.append(joongsung)
        result.append(jongsung)

    return "".join(result)


In [7]:
# 각 문장을 jamo_split하여 저장
#from JamoSplit import jamo_split, jamo_combine
tmp.loc[:,'댓글'] = tmp.loc[:,'댓글'].apply(lambda x: jamo_split(x))
tmp.loc[:,'댓글'] = tmp.loc[:,'댓글'].apply(lambda x: x.split(" "))

In [8]:
tmp.sample(15)

,댓글,비속어여부
34245,"[ㅇㅏ_ㅅㅡ_ㅍㅣ_ㄹㅣㄴ, ㄱㅜㄱㄴㅐ_ㅍㅏㄴㅁㅐ_, ㄱㅡㅁㅈㅣ_ㅇㅏ_ㄴㅣ_ㄴㅑ_]",1
18306,"[ㄷㅡ_ㄹㅜ_ㅋㅣㅇㄸㅐ_ㅁㅜㄴㅇㅔ_, ㅈㅓㅂㅇㅓㅆㄷㅓㄴ, ㅇㅘ_ㅇㅜ_ㄷㅏ_ㅅㅣ_ㅎㅏ...",1
5915,[ㅃㅏ_ㄱㅜ_ㄹㅣ_],1
154745,"[ㄱㅑ_ㄴㅡㄴ, ㄱㅏㄹㅋㅕ_, ㅈㅝ_ㄷㅗ_, ㅁㅗㅅ, ㅇㅏㄹㅇㅏ_, ㅊㅕ_ㅁㅓㄱㄴㅡ...",1
106893,"[ㄱㅗㄷ, ㅇㅣㅁㅈㅣㄴㅇㅙ_ㄹㅏㄴㄸㅐ_, ㅈㅗ_ㅅㅏㅇ, ㅈㅜㄱㅇㅓㅆㄷㅏ_ㄱㅗ_, ㅂ...",1
161059,"[ㄱㅡ_ㄴㅑㅇ, ㄱㅏ_ㅂㅕㅂㄱㅔ_, ㅂㅗ_ㅁㅕㄴ, ㄷㅚㅁ]",0
188003,"[ㅇㅓ_ㅊㅏ_ㅍㅣ_, ㅈㅏ_ㅎㅏㄴㄷㅏㅇㄷㅗ_, ㅁㅣㄴㅈㅜ_ㄷㅏㅇ, 2ㅈㅜㅇㄷㅐ_ㅇㅣ...",1
129382,[ㅇㅣ_ㅈㅓㅇㄷㅗ_ㅇㅢ_ㅍㅕㅇㅈㅓㅁㅇㅡㄴㅇㅏ_ㄴㅣㄴㄷㅔ_ㄴㅏ_ㄹㅡㅁㅈㅐ_ㅁㅣㅆㄴㅡㄴ...,0
40506,"[ㅍㅏ_ㄱㅕㄱㅈㅓㄱㅇㅣㄴ, ㅅㅗ_ㅈㅐ_ㅇㅢ_, ㅇㅣ_ㅇㅑ_ㄱㅣ_ㅇㅕㅆㄷㅏ_]",0
39233,"[ㅇㅣ_ㄹㅓㅎㄱㅔ_, ㅂㅕㅇㅁㅏㅅㅅㅡ_ㄹㅓ_ㅇㅜㄴ, ㅍㅏㄴㅌㅏ_ㅈㅣ_, ㅅㅏ_ㄱㅡㄱ...",0


In [9]:
# dataframe -> list로 변환
sentence_list = list(tmp['댓글'])
len(sentence_list)

190581

In [10]:
# fasttext 인풋 완성!
sentence_list[:10]

[['ㅈㅘ_ㅂㅐ_', 'ㄲㅏ_ㄴㅡㄴㄱㅓㄴ'],
 ['ㅈㅣㅂㅇㅔ_',
  'ㄹㅗㅇ',
  'ㅍㅐ_ㄷㅣㅇㅁㅏㄴ',
  'ㅅㅔ_',
  'ㄱㅐ_ㄷㅏ_',
  '10ㄴㅕㄴ',
  'ㄷㅓ_',
  'ㅇㅣㅂㅇㅓ_ㅇㅑ_ㅈㅣ_'],
 ['ㄱㅐ_ㅅㅗ_ㄹㅣ_ㅇㅑ_',
  'ㄴㅣ_ㄱㅏ_',
  'ㅃㅏㄹㄱㅐㅇㅇㅣ_ㄹㅡㄹ',
  'ㅇㅗㅇㅎㅗ_ㅎㅏ_ㄱㅗ_',
  'ㄷㅡ_ㄹㅜ_ㅋㅣㅇㅇㅡㄹ',
  'ㅈㅣㅅㅇㅣ_ㄹㅏ_ㄱㅗ_',
  'ㅁㅏㄹㅁㅗㅅㅎㅐ_ㅅㅓ_',
  'ㅃㅣ_ㅈㅣㄴㄱㅓ_ㅇㅑ_',
  'ㅃㅏㄹㄱㅐㅇㅇㅏ_'],
 ['ㅅㅔ_ㅌㅏㄱㅇㅣ_ㄹㅏ_ㄱㅗ_', 'ㅂㅘ_ㄷㅗ_', 'ㄷㅚㄴㄷㅏ_'],
 ['ㅇㅐ_ㅅㅐ_ㄲㅣ_ㄱㅏ_', 'ㅊㅗ_ㄷㅣㅇㄷㅗ_', 'ㅇㅏ_ㄴㅣ_ㄱㅗ_'],
 ['731ㅂㅜ_ㄷㅐ_ㅇㅢ_',
  'ㅎㅜ_ㅇㅖ_ㄹㅏ_',
  'ㄱㅡ_ㄹㅓㄴㅈㅣ_',
  'ㄱㅏ_ㅎㅏㄱㅈㅓㄱㅇㅣㄴ',
  'ㅇㅏ_ㅇㅣ_ㄷㅣ_ㅇㅓ_ㄴㅡㄴ',
  'ㅅㅔ_ㄱㅖ_ㅊㅚ_ㄱㅗ_ㅇㅣㅁ',
  'ㅇㅣ_ㄹㅐ_ㅅㅓ_',
  'ㅇㅐ_ㄱㅛ_ㅁㅏㄴ',
  'ㄸㅓㄹㅇㅓ_ㄷㅗ_',
  'ㄷㅗㄴ',
  'ㅂㅓㄹㄹㅣ_ㄴㅡㄴ',
  'ㅎㅏㄴㄱㅜㄱㅇㅔ_',
  'ㄱㅣ_ㄹㅡㄹ',
  'ㅆㅓ_ㅅㅓ_',
  'ㅈㅣㄴㅊㅜㄹㅎㅏ_ㄹㅕ_ㄱㅗ_',
  'ㅎㅏ_ㅈㅣ_ㅈㅗ_ㅅㅔㄴㄴㅏㅁㅈㅏ_ㄷㅡㄹㅇㅡㄴ',
  'ㄸㅗ_',
  'ㅇㅣ_ㅃㅡㄴㅇㅕ_ㅈㅏ_ㅁㅏㄴ',
  'ㅂㅗ_ㅁㅕㄴ',
  'ㅅㅏ_ㅈㅗㄱㅇㅡㄹ',
  'ㅁㅗㅅㅆㅡ_ㅁㅕ_',
  'ㄱㅗㅇㅈㅜ_ㄷㅐ_ㅈㅓㅂㅎㅐ_ㅈㅜ_ㄴㅡㄴ',
  'ㄴㅗㅁㄷㅡㄹㅇㅣ_ㄴㅣ_'],
 ['ㅈㅐ_ㅇㅏㅇㅇㅣ_ㅎㅏㄴㄱㅓㄴㅎㅐㅅㄴㅗ_'],
 ['ㄱㅡㄹㅆㅡㄴㅇㅣ_',
  'ㅇㅘ_ㄲㅜ_',
  'ㅅㅡㅇㄹㅣ_ㅇㅔ_',
  'ㅂㅣ_ㅎㅏ_ㅁㅕㄴ',
  'ㅂㅏㅇㅅㅏ_ㄴㅡㅇ',
  'ㅍㅣ_ㅍㅗㄱ',
  'ㅇㅝㄴㅅㅜㅇㅇㅣ_',
  'ㅇㅣㄹㄷㅡㅅ'],
 ['ㅁㅏ_',
  'ㅆㅣ_ㅂㅏㄹㄹㅕㄴ',
  'ㅇㅏ_',
  'ㅁㅕㅊㅍㅕㅇㅇㅣ_ㄱㅗ_',
  'ㅁㅐㅅㄱㅐ_ㄷㅡ_ㄱㅏㅆㄴㅗ_',
  'ㄴㅣ_',
  'ㄷㅐ_ㅎㅏ_ㅇㅣ_ㅎㅐㅁㅎㅏ_ㄱㅗ_',
  'ㅎㅐ_ㅂ

### FastText WordEmbedding

In [20]:
# fasttext 적용
from gensim.models import FastText
# 임베딩 차원: 100
# window size: 좌우 2단어 비속어는 좌우단어와 별로 연관이 없다고 판단...
# min_count: 최소 3번 등장한 단어들
# workers: -1 전부!!
# sg: skipgram이 더 성능이 좋기 때문
# min_n max_n : n-gram단위인데 한글자가 3글자라 최소 자모3개부터 최대 6개까지 ngram하기로 하였다. 1글자 ~ 2글자
# iter: 반복횟수 10
model = FastText(sentence_list, size=100, window=2, min_count=3, workers=4, sg=1, min_n=3, max_n=6, iter=10)

In [21]:
# 총 62105개 단어로 vocab이 만들어졌다.
# len(model.wv)
len(model.wv.vocab)

62105

In [22]:
# 연도 같은 경우는 연도와 비슷한 단어들이 나온다.
model.wv.most_similar(jamo_split("2018년"))

[('2017ㄴㅕㄴ', 0.9939414262771606),
 ('2019ㄴㅕㄴ', 0.9929779171943665),
 ('2016ㄴㅕㄴ', 0.9876396656036377),
 ('2011ㄴㅕㄴ', 0.9789131283760071),
 ('18ㄴㅕㄴ', 0.9764711856842041),
 ('2012ㄴㅕㄴ', 0.9757429361343384),
 ('1988ㄴㅕㄴ', 0.9710288047790527),
 ('1982ㄴㅕㄴ', 0.9700126051902771),
 ('2002ㄴㅕㄴ', 0.9699717164039612),
 ('17ㄴㅕㄴ', 0.9687004089355469)]

In [23]:
# 욕설 같은 경우는 비슷한 형태의 욕설이 나온다.
model.wv.most_similar(jamo_split("개새끼"))

[('ㅈㅟ_ㅅㅐ_ㄲㅣ_', 0.9744609594345093),
 ('ㄱㅐ_ㅆㅣㅂㅅㅐ_ㄲㅣ_', 0.9732168912887573),
 ('ㅊㅣ_ㅁㅐ_ㅅㅐ_ㄲㅣ_', 0.9710700511932373),
 ('ㅇㅐ_ㅅㅐ_ㄲㅣ_', 0.9687486886978149),
 ('ㅉㅏㅇㄲㅐ_ㅅㅐ_ㄲㅣ_', 0.9680942893028259),
 ('ㅅㅣㅂㅅㅐ_ㄲㅣ_', 0.9667065739631653),
 ('ㄱㅐ_ㄷㅗㄱㅅㅐ_ㄲㅣ_', 0.9656611680984497),
 ('ㅊㅣ_ㅌㅏ_ㅅㅐ_ㄲㅣ_', 0.9641475677490234),
 ('ㅆㅣㅂㅅㅐ_ㄲㅣ_', 0.9631043076515198),
 ('ㅆㅣ_ㅂㅏㄹㄱㅐ_ㅅㅐ_ㄲㅣ_', 0.9628869891166687)]

In [24]:
model.wv.most_similar(jamo_split("시발"))

[('ㅁㅝㄴㅅㅣ_ㅂㅏㄹ', 0.9585663080215454),
 ('ㅆㅣ_ㅂㅏㄹ', 0.9494709372520447),
 ('ㅅㅣ_ㅂㅏㅋ', 0.9484204053878784),
 ('ㅅㅣ_ㅂㅏㄹㄴㅓㅁ', 0.9470741152763367),
 ('ㅅㅣ_ㅂㅏㄹㄲㅓ_', 0.9322314262390137),
 ('ㅅㅣ_ㅂㅏㄹㅇㅘ_', 0.9310782551765442),
 ('ㅇㅘ_ㅅㅣ_ㅂㅏㄹ', 0.9289675951004028),
 ('ㅇㅘ_ㅆㅣ_ㅂㅏㄹ', 0.9286604523658752),
 ('ㅅㅣ_ㅂㅏㄹㄴㅗㅁ', 0.9272249937057495),
 ('ㅅㅣ_ㅂㅏㄹㄹㅕㄴ', 0.9139248132705688)]

In [26]:
# 시발점을 쳤더니.. 아주 관련없는 단어들이 뜬다. 뭐지?
model.wv.most_similar(jamo_split("시바"))

[('ㅅㅣ_ㅂㅏㅋ', 0.9346566796302795),
 ('ㅅㅣ_ㅂㅏㄹㄴㅓㅁ', 0.9328367710113525),
 ('ㅅㅣ_ㅂㅏㄹㄹㅓㅁㅇㅏ_', 0.9319655895233154),
 ('ㅅㅣ_ㅂㅏㄹㄹㅗㅁㅇㅏ_', 0.9312819838523865),
 ('ㅅㅣ_ㅂㅏㄹㄲㅓ_', 0.9232179522514343),
 ('ㅅㅣ_ㅂㅏㄹㄴㅗㅁ', 0.9213518500328064),
 ('ㅅㅣ_ㅂㅏㄹㅇㅏ_', 0.9189701676368713),
 ('ㅅㅣ_ㅂㅏㄹㄹㅕㄴㅇㅏ_', 0.9157925844192505),
 ('ㅅㅣ_ㅂㅏㄹㄴㅕㄴㅇㅏ_', 0.9128421545028687),
 ('ㅅㅣ_ㅂㅏㄹㄹㅕㄴ', 0.9093796610832214)]

In [27]:
model.wv.most_similar(jamo_split("새끼"))

[('ㅅㅐ_ㄲㅣㅋ', 0.9636362195014954),
 ('ㅈㅟ_ㅅㅐ_ㄲㅣ_', 0.9523922204971313),
 ('ㅅㅐ_ㄲㅣ_ㅇㅑㅋ', 0.9516310691833496),
 ('ㅇㅐ_ㅅㅐ_ㄲㅣ_', 0.9502369165420532),
 ('ㅅㅣㅂㅅㅐ_ㄲㅣ_', 0.9416285157203674),
 ('ㅆㅣㅂㅅㅐ_ㄲㅣ_', 0.9389625787734985),
 ('ㅁㅐ_ㄲㅣ_', 0.9385172128677368),
 ('ㅅㅐ_ㄲㅣㄴ', 0.9383877515792847),
 ('ㅇㅔㅁㅊㅏㅇㅅㅐ_ㄲㅣ_', 0.938178300857544),
 ('ㅆㅣㅂㅆㅐ_ㄲㅣ_', 0.9344974756240845)]

In [28]:
model.wv.most_similar(jamo_split("존나"))

[('ㅈㅗㄴㄴㅏ_ㅍㅐㅁ', 0.9659515619277954),
 ('ㅅㅗㄴㄴㅏ_', 0.9243826866149902),
 ('ㅈㅗ_ㅇㅗㄴㄴㅏ_', 0.9089362025260925),
 ('ㅈㅗㄴㄴㅏ_ㅎㅏㅁ', 0.8903926014900208),
 ('ㅈㅗㄴㄴㅏ_ㅇㅜㅅㅈㅏ_', 0.8892310857772827),
 ('ㅈㅛㄴㄴㅏ_', 0.8631578683853149),
 ('ㅈㅝㄴㄴㅏ_', 0.8607434034347534),
 ('ㅇㅛㄴㄴㅏ_', 0.8575743436813354),
 ('ㅈㅗㄴㄴㅏ_ㅁㅏㄶㄴㅔ_', 0.8511917591094971),
 ('ㅈㅗㄴㄴㅏ_ㅋㅡ_ㄴㅔ_', 0.8501119613647461)]

In [29]:
model.wv.most_similar(jamo_split("지랄"))

[('ㅈㅣ_ㄹㅏㄹㅂㅕㅇ', 0.9671280980110168),
 ('ㅈㅣ_ㄹㅏㄹㅈㅗㅁ', 0.9590924978256226),
 ('ㅈㅣ_ㄹㅏㄹㅂㅏㄹㄱㅘㅇ', 0.9504424929618835),
 ('ㅈㅣ_ㄹㅏㄹㅁㅏ_', 0.9439529180526733),
 ('ㅈㅗㅈㅈㅣ_ㄹㅏㄹ', 0.9332311749458313),
 ('ㅈㅣ_ㄹㅏㄹㅇㅣㅁ', 0.9314278364181519),
 ('ㅈㅣ_ㄹㅏㄹㅎㅏㅁ', 0.928388774394989),
 ('ㅂㅕㄹㅈㅣ_ㄹㅏㄹ', 0.9173683524131775),
 ('ㅈㅣ_ㄹㅏㄹㅎㅏㄹ', 0.9025567770004272),
 ('ㄱㅐ_ㅈㅣ_ㄹㅏㄹ', 0.8838884234428406)]

In [30]:
model.wv.most_similar(jamo_split("벌레"))

[('ㅆㅣㅂㄱㅓㄹㄹㅔ_', 0.9583634734153748),
 ('ㅎㅔ_ㅂㅓㄹㄹㅔ_', 0.9522801041603088),
 ('ㄱㅓㄹㄹㅔ_', 0.9474725723266602),
 ('ㅉㅏㅇㅋㅟ_ㅂㅓㄹㄹㅔ_', 0.9165828227996826),
 ('ㅊㅕ_ㅁㅓㄱㄴㅔ_', 0.8955965638160706),
 ('ㅂㅏ_ㅋㅟ_ㅂㅓㄹㄹㅔ_', 0.8930861353874207),
 ('ㅆㅣㅂㅅㅔ_', 0.8784026503562927),
 ('ㄱㅓㄹㄹㅕㅆㄴㅔ_', 0.8756577968597412),
 ('ㅁㅓㄱㄴㅔ_', 0.8675596117973328),
 ('ㄲㅏ_ㅍㅔ_', 0.862654447555542)]

In [31]:
# # embedding model 저장
# model.save("./gensim_festtext.model_1")

In [53]:
embedding_model.wv.most_similar(jamo_split("멍청이"))

[('ㅁㅓㅇㅁㅓㅇㅇㅣ_', 0.9608525037765503),
 ('ㄲㅐㅇㄲㅐㅇㅇㅣ_', 0.9559065103530884),
 ('ㅈㅗㅈㄴㅑㅇㅇㅣ_', 0.9547886848449707),
 ('ㄸㅜㅇㄸㅐㅇㅇㅣ_', 0.9533778429031372),
 ('ㅇㅛㄱㅈㅐㅇㅇㅣ_', 0.9515029191970825),
 ('ㅇㅑㄱㅈㅐㅇㅇㅣ_', 0.9506490230560303),
 ('ㅇㅑㅇㅇㅣ_', 0.9498114585876465),
 ('ㅇㅡㄴㅎㅐㅇㅇㅣ_', 0.9482777118682861),
 ('ㅃㅐㅇㅃㅐㅇㅇㅣ_', 0.9477080702781677),
 ('ㄸㅗㅇㅇㅣ_', 0.9474722146987915)]

### ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [47]:
# fasttext 모델 불러오기

from gensim.models import FastText

embedding_model = FastText.load("./gensim_festtext.model_1")

In [13]:
# 각 단어를 벡터화 시켜주는 과정 3 x 100(embedding dimension) 
tmp['댓글'] = tmp['댓글'].apply(lambda x: [embedding_model[_] for _ in x])

/tmp/ipykernel_24033/1830680873.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  tmp['댓글'] = tmp['댓글'].apply(lambda x: [embedding_model[_] for _ in x])


In [14]:
# 벡터화된 데이터로 저장
tmp.to_json("./fast-text_vectorized_labeled_data_1.json")

## 패딩하기
#### 문장마다 단어의 개수가 다르기 때문에 한 문장은 벡터의 개수가 전부 다르다. 벡터의 개수가 다를 경우 LSTM 모델 학습할 때 에러가 발생하기때문에 모든 데이터의 벡터의 개수를 맞춰 0으로 채워주는 패딩 작업 필요

In [4]:
data = pd.read_json("./fast-text_vectorized_labeled_data_1.json")
data.columns = ["댓글", "비속어여부"]
data.head(10)

,댓글,비속어여부
0,"[[-0.7201249599, 0.301058948, -0.4723067284000...",1
1,"[[-0.37243309620000004, 0.6573367715, 0.334138...",0
2,"[[-0.9452651143, 1.1995326281, -0.6718460917, ...",1
3,"[[-1.0153485537, 0.6960261464, 0.7171273828, -...",0
4,"[[-0.6345767379, 1.1871398687, -0.1858331263, ...",1
5,"[[0.12848372760000001, -0.1863826066, -0.03566...",1
6,"[[-0.6168811321000001, 0.6517000198, 0.0483778...",1
7,"[[-1.2506676911999999, 0.8527351022, -0.042807...",1
8,"[[0.24690280850000002, 0.9139938951000001, -0....",1
9,"[[-0.40946820380000004, 0.4967239201, 0.268821...",0


In [60]:
# 상위 50개 단어 개수 살펴보기
for i in range(0,51):
    print(len(data.iloc[i,0]))

2
8
9
3
3
23
1
8
8
9
7
13
5
7
8
4
8
19
4
2
7
20
3
8
5
4
6
3
1
15
4
2
2
1
8
3
4
2
44
13
5
3
2
7
16
2
7
17
2
3
5


In [5]:
print('문장의 최대 단어길이 :',max(len(review) for review in data['댓글']))
print('문장의 평균 단어길이 :',sum(map(len, data['댓글']))/len(data['댓글']))

# 한 문장의 최대 단어의 개수는 312개, 평균 단어의 개수는 8개 .. 이거맞아?

문장의 최대 단어길이 : 312
문장의 평균 단어길이 : 8.244289829521305


In [6]:
def below_threshold_len(max_len, nested_list):
    count = 0
    for sentence in nested_list:
        if(len(sentence) <= max_len):
            count = count + 1
    print('전체 문장들 중 단어개수가 %s 이하인 문장의 비율: %s'%(max_len, (count / len(nested_list))*100))
    
max_len = 30
below_threshold_len(max_len, data['댓글'])

# 97 % 가 30 단어 이하이기 때문에 최대단어수는 30으로 패딩해주는것이 좋을것같다.

전체 문장들 중 단어개수가 30 이하인 문장의 비율: 97.1733803474638


In [7]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(data['댓글'], data['비속어여부'] , test_size=0.2, random_state=0)
print(len(train_x), len(train_y), len(test_x), len(test_y))

152464 152464 38117 38117


In [8]:
data['댓글'].head(10)

0    [[-0.7201249599, 0.301058948, -0.4723067284000...
1    [[-0.37243309620000004, 0.6573367715, 0.334138...
2    [[-0.9452651143, 1.1995326281, -0.6718460917, ...
3    [[-1.0153485537, 0.6960261464, 0.7171273828, -...
4    [[-0.6345767379, 1.1871398687, -0.1858331263, ...
5    [[0.12848372760000001, -0.1863826066, -0.03566...
6    [[-0.6168811321000001, 0.6517000198, 0.0483778...
7    [[-1.2506676911999999, 0.8527351022, -0.042807...
8    [[0.24690280850000002, 0.9139938951000001, -0....
9    [[-0.40946820380000004, 0.4967239201, 0.268821...
Name: 댓글, dtype: object

In [9]:
train_x

46726     [[-0.3872094452, -0.16645319760000002, -0.1413...
94165     [[-0.42452299590000003, 0.5644479394, -0.18073...
136954    [[-1.252489686, -0.6533976197, 1.1044143438, 0...
183719    [[-0.08469905700000001, 0.428263098, 0.5940571...
133146    [[0.3221236169, 0.8550377488, -0.1347977668, 0...
                                ...                        
152315    [[-0.47730064390000004, -0.0412694402, -0.6019...
176963    [[0.1267471761, -0.30235001440000003, 0.022919...
117952    [[-0.6393437386, 0.0126002952, -0.3966500163, ...
173685    [[-0.1671716422, -0.6286797523000001, -0.39739...
43567     [[-0.1596149653, 0.7296592593, -0.467102944900...
Name: 댓글, Length: 152464, dtype: object

In [14]:
# y array형식으로 변경
y_train = np.array(train_y)
y_test = np.array(test_y)

In [10]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(train_x, maxlen=max_len)
X_test = pad_sequences(test_x, maxlen=max_len)

In [11]:
print(X_train.shape)

(152464, 30, 100)


In [12]:
print(X_test.shape)

(38117, 30, 100)


### LSTM 학습하기 

In [15]:
from tensorflow.python.keras.layers import Embedding, Dense, LSTM
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(LSTM(units=1, input_shape=(30, 100)))
model.add(Dense(1, activation='sigmoid'))



# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
# mc = ModelCheckpoint('FT_best_model_v0.0.1.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)


model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

2022-05-10 08:43:50.035205: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-10 08:43:50.076834: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0001:00:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-05-10 08:43:50.078838: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-05-10 08:43:50.195343: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-05-10 08:43:50.248895: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-05-10 08:43:50.284837: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-05-

Epoch 1/5
4758/4765 [============================>.] - ETA: 0s - loss: 0.3592 - accuracy: 0.8595

2022-05-10 08:44:19.678602: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 457404000 exceeds 10% of free system memory.


4765/4765 [==============================] - 29s 6ms/step - loss: 0.3591 - accuracy: 0.8595 - val_loss: 0.2957 - val_accuracy: 0.8750
Epoch 2/5
4764/4765 [============================>.] - ETA: 0s - loss: 0.2844 - accuracy: 0.8787

2022-05-10 08:44:48.841744: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 457404000 exceeds 10% of free system memory.


4765/4765 [==============================] - 29s 6ms/step - loss: 0.2844 - accuracy: 0.8787 - val_loss: 0.2833 - val_accuracy: 0.8782
Epoch 3/5
4762/4765 [============================>.] - ETA: 0s - loss: 0.2790 - accuracy: 0.8794

2022-05-10 08:45:17.700566: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 457404000 exceeds 10% of free system memory.


4765/4765 [==============================] - 29s 6ms/step - loss: 0.2789 - accuracy: 0.8794 - val_loss: 0.2812 - val_accuracy: 0.8790
Epoch 4/5
4758/4765 [============================>.] - ETA: 0s - loss: 0.2777 - accuracy: 0.8801

2022-05-10 08:45:46.484933: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 457404000 exceeds 10% of free system memory.


4765/4765 [==============================] - 29s 6ms/step - loss: 0.2777 - accuracy: 0.8801 - val_loss: 0.2807 - val_accuracy: 0.8785
Epoch 5/5
4765/4765 [==============================] - 29s 6ms/step - loss: 0.2773 - accuracy: 0.8798 - val_loss: 0.2804 - val_accuracy: 0.8787


In [25]:
model.evaluate(X_test, y_test)

1192/1192 [==============================] - 2s 2ms/step - loss: 0.2804 - accuracy: 0.8787


[0.28039073944091797, 0.8786892890930176]

In [26]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

1192/1192 [==============================] - 2s 2ms/step - loss: 0.2804 - accuracy: 0.8787

 테스트 정확도: 0.8787


In [27]:
model.save("./fasttext_1.h5")

In [28]:
model.save("./fasttext_model")

2022-05-10 09:01:21.980801: W tensorflow/python/util/util.cc:329] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./fasttext_model/assets


In [30]:
loaded_model = load_model('./fasttext_1.h5')

## 예측하기 - 미완성

In [57]:
def test_result(jamo_split, loaded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(sentence_number):
        if index < len(test_word_split):
            fast_vec.append(loaded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] == 0:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")


In [60]:
s = '비속어가 포함된 문장'
for s_split in s.split():
    test_word_run = s_split
    result = loaded_model(jamo_split(test_word_run))
    count = 0
    for _, word_temp in result:
        for w in word_list:
            if w in run_inverse(word_temp):
                count+=1


AttributeError: 'str' object has no attribute '_keras_mask'

In [39]:
test_result(*loaded_model.test('가나다라마바사'))
print(model.predict("가나다라마바사"))

AttributeError: 'Sequential' object has no attribute 'test'

In [36]:
from gensim.models import FastText

model = fasttext.train_supervised('cooking.stackexchange.txt', wordNgrams=2, epoch=25, lr=0.5)

# model = fasttext.load_model("model_cooking.bin")

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('cooking.stackexchange.txt'))

print (model.predict("Which baking dish is best to bake a banana bread ?"))
# 3개의 카테고리 정보를 함께 출력
# print (model.predict("Which baking dish is best to bake a banana bread ?", k=3))

model.save_model("model_cooking.bin")

In [45]:
test_word = '시발'
test_word_run = jamo_split(test_word)
result= loaded_model.predict(test_word_run)
for _,word_temp in result:
    print(run_inverse(word_temp))

IndexError: list index out of range